In [ ]:
Create a test database and table with one million dummy records
============================================================================

In [ ]:

CREATE DATABASE loadtestdb;

use loadtestdb;

CREATE TABLE loadtesttable (
ID INT PRIMARY KEY AUTO_INCREMENT,
Name VARCHAR(255),
Age INT,
Salary DECIMAL(10, 2),
Department VARCHAR(50),
City VARCHAR(100),
Country VARCHAR(100)
);


INSERT INTO loadtesttable (Name, Age, Salary, Department, City, Country)
SELECT
    CONCAT(CHAR(FLOOR(RAND() * 26) + 65), 'Person', n),
    FLOOR(RAND() * 100) + 18,
    ROUND(RAND() * 10000000, 2),
    CASE WHEN RAND() < 0.5 THEN 'IT' ELSE 'Sales' END,
    CASE WHEN RAND() < 0.5 THEN 'New York' ELSE 'Los Angeles' END,
    CASE WHEN RAND() < 0.5 THEN 'USA' ELSE 'Canada' END
FROM (
    SELECT
        a.N + b.N * 10 + c.N * 100 + d.N * 1000 + e.N * 10000 + f.N * 100000 + 1 AS n
    FROM
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) a,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) b,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) c,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) d,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) e,
        (SELECT 0 AS N UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9) f
) AS Numbers;

Query OK, 1000000 rows affected (14.62 sec)
Records: 1000000  Duplicates: 0  Warnings: 0


In [ ]:

Server CPU and Memory Info 
=================================================

[root@mysqlprod02 ~]# lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  1
Socket(s):           4


[root@mysqlprod02 ~]# free -m

              total        used        free      shared  buff/cache   available
Mem:           5709        1599        3066          36        1043        3835
Swap:          3071           0        3071
[root@mysqlprod02 ~]# 

In [ ]:

Run tests with default values 
===========================================================

In [ ]:


mysql> show variables like '%innodb_redo_log%';
+------------------------------+-----------+
| Variable_name                | Value     |
+------------------------------+-----------+
| innodb_redo_log_capacity     | 104857600 |
+------------------------------+-----------+

mysql> show variables like '%innodb_buffer_pool%';
+-------------------------------------+----------------+
| Variable_name                       | Value          |
+-------------------------------------+----------------+
| innodb_buffer_pool_chunk_size       | 134217728      |
| innodb_buffer_pool_instances        | 1              |
| innodb_buffer_pool_size             | 134217728      |
+-------------------------------------+----------------+



To run a concurrency test with 5, 10, and 15 threads, each executing 50 queries 5 times, run the following command
============================================================================================================================


[root@mysqlprod02 ~]# mysqlslap --concurrency=5,10,15 --iterations=5 --number-of-queries=50 
                      --query="SELECT ID, Name, Age, Salary, Department, City, Country FROM loadtesttable WHERE Name like 'A%' AND 
                      Age BETWEEN 30 AND 40;" --create-schema=loadtestdb --verbose
					  
Benchmark
	Average number of seconds to run all queries: 15.041 seconds
	Minimum number of seconds to run all queries: 12.847 seconds
	Maximum number of seconds to run all queries: 19.217 seconds
	Number of clients running queries: 5
	Average number of queries per client: 10

Benchmark
	Average number of seconds to run all queries: 14.087 seconds
	Minimum number of seconds to run all queries: 12.908 seconds
	Maximum number of seconds to run all queries: 15.710 seconds
	Number of clients running queries: 10
	Average number of queries per client: 5

Benchmark
	Average number of seconds to run all queries: 13.244 seconds
	Minimum number of seconds to run all queries: 11.588 seconds
	Maximum number of seconds to run all queries: 15.143 seconds
	Number of clients running queries: 15
	Average number of queries per client: 3


top - 01:55:59 up 13 min,  1 user,  load average: 1.20, 0.43, 0.24
Tasks: 257 total,   2 running, 255 sleeping,   0 stopped,   0 zombie
%Cpu(s): 97.6 us,  0.6 sy,  0.0 ni,  0.9 id,  0.0 wa,  0.8 hi,  0.1 si,  0.0 st
MiB Mem :   5709.6 total,   2854.6 free,   1710.7 used,   1144.3 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3714.5 avail Mem 


top - 01:56:29 up 13 min,  1 user,  load average: 3.28, 1.01, 0.44
Tasks: 257 total,   1 running, 256 sleeping,   0 stopped,   0 zombie
%Cpu(s): 97.8 us,  0.8 sy,  0.0 ni,  0.0 id,  0.0 wa,  1.2 hi,  0.2 si,  0.0 st
MiB Mem :   5709.6 total,   2854.3 free,   1711.0 used,   1144.3 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3714.2 avail Mem 



To run a stress test with 20 threads, each executing the query 10 times, run the following command:
--------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=20 --iterations=10 --query="SELECT ID, Name, Age, Salary, Department, City, Country FROM
                                (SELECT *, ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) AS R FROM loadtesttable) AS 
                                ranked WHERE R <= 5;" --create-schema=loadtestdb --verbose

Benchmark
	Average number of seconds to run all queries: 61.943 seconds
	Minimum number of seconds to run all queries: 50.947 seconds
	Maximum number of seconds to run all queries: 101.671 seconds
	Number of clients running queries: 20
	Average number of queries per client: 1


top - 02:05:19 up 22 min,  1 user,  load average: 8.93, 4.05, 2.42
Tasks: 255 total,   4 running, 251 sleeping,   0 stopped,   0 zombie
%Cpu(s): 29.5 us, 67.2 sy,  0.0 ni,  0.0 id,  0.0 wa,  3.0 hi,  0.3 si,  0.0 st
MiB Mem :   5709.6 total,   1387.2 free,   1753.0 used,   2569.4 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3670.4 avail Mem 


top - 02:06:02 up 23 min,  1 user,  load average: 16.61, 6.65, 3.37
Tasks: 256 total,   3 running, 253 sleeping,   0 stopped,   0 zombie
%Cpu(s): 84.8 us,  8.1 sy,  0.0 ni,  1.3 id,  0.6 wa,  4.2 hi,  0.9 si,  0.0 st
MiB Mem :   5709.6 total,   1456.0 free,   2095.4 used,   2158.1 buff/cache
MiB Swap:   3072.0 total,   3071.5 free,      0.5 used.   3335.8 avail Mem 


top - 02:06:54 up 23 min,  1 user,  load average: 17.84, 8.58, 4.21
Tasks: 257 total,   3 running, 254 sleeping,   0 stopped,   0 zombie
%Cpu(s): 26.0 us, 34.9 sy,  0.0 ni,  0.0 id, 17.1 wa, 21.0 hi,  1.0 si,  0.0 st
MiB Mem :   5709.6 total,    568.3 free,   1991.9 used,   3149.4 buff/cache
MiB Swap:   3072.0 total,   3070.2 free,      1.8 used.   3438.3 avail Mem 



To run a benchmark test with 10 threads, each executing 1000 randomly generated queries with a mixed load type,
 run the following command:
 ----------------------------------------------------------------------------------------------------------------


[root@mysqlprod02 ~]# mysqlslap --concurrency=10 --iterations=1 --number-of-queries=100000 --auto-generate-sql 
                                --auto-generate-sql-load-type=MIXED --verbose

Benchmark
	Average number of seconds to run all queries: 669.630 seconds
	Minimum number of seconds to run all queries: 669.630 seconds
	Maximum number of seconds to run all queries: 669.630 seconds
	Number of clients running queries: 10
	Average number of queries per client: 10000


top - 12:29:26 up  1:20,  1 user,  load average: 6.17, 3.07, 2.17
Tasks: 256 total,   2 running, 254 sleeping,   0 stopped,   0 zombie
%Cpu(s): 51.1 us, 30.3 sy,  0.0 ni, 14.6 id,  0.7 wa,  2.9 hi,  0.4 si,  0.0 st
MiB Mem :   5709.6 total,   1986.6 free,   1921.5 used,   1801.4 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3500.1 avail Mem 


top - 12:29:38 up  1:20,  1 user,  load average: 8.04, 3.64, 2.37
Tasks: 256 total,   3 running, 253 sleeping,   0 stopped,   0 zombie
%Cpu(s): 52.7 us, 29.9 sy,  0.0 ni, 13.3 id,  0.7 wa,  3.1 hi,  0.3 si,  0.0 st
MiB Mem :   5709.6 total,   1984.3 free,   1922.0 used,   1803.3 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3499.6 avail Mem 


top - 12:31:22 up  1:22,  1 user,  load average: 12.38, 6.56, 3.57
Tasks: 257 total,   2 running, 255 sleeping,   0 stopped,   0 zombie
%Cpu(s): 60.5 us, 30.5 sy,  0.0 ni,  6.8 id,  0.2 wa,  1.9 hi,  0.1 si,  0.0 st
MiB Mem :   5709.6 total,   1965.6 free,   1932.9 used,   1811.0 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3488.7 avail Mem 


In [ ]:
Run tests with setting innodb dedicated server
===========================================================

In [ ]:

[root@mysqlprod02 ~]# cat /etc/my.cnf | grep dedicated
innodb-dedicated-server = TRUE
[root@mysqlprod02 ~]# 

[root@mysqlprod02 ~]# systemctl restart mysqld
[root@mysqlprod02 ~]# 

mysql> show variables like '%innodb_flush_method%';
+---------------------+-------------------+
| Variable_name       | Value             |
+---------------------+-------------------+
| innodb_flush_method | O_DIRECT_NO_FSYNC |
+---------------------+-------------------+

mysql> show variables like '%innodb_redo_log%';
+------------------------------+------------+
| Variable_name                | Value      |
+------------------------------+------------+
| innodb_redo_log_capacity     | 2684354560 |
+------------------------------+------------+

mysql> show variables like '%innodb_buffer_pool%';
+-------------------------------------+----------------+
| Variable_name                       | Value          |
+-------------------------------------+----------------+
| innodb_buffer_pool_chunk_size       | 134217728      |
| innodb_buffer_pool_instances        | 8              |
| innodb_buffer_pool_size             | 5368709120     |
+-------------------------------------+----------------+


To run a concurrency test with 5, 10, and 15 threads, each executing 50 queries 5 times, run the following command
============================================================================================================================


[root@mysqlprod02 ~]# mysqlslap --concurrency=5,10,15 --iterations=5 --number-of-queries=50 
                                --query="SELECT ID, Name, Age, Salary, Department, City, Country FROM loadtesttable WHERE Name like 
								 'A%' AND Age BETWEEN 30 AND 40;" --create-schema=loadtestdb --verbose
								 
Benchmark
	Average number of seconds to run all queries: 13.879 seconds
	Minimum number of seconds to run all queries: 12.573 seconds
	Maximum number of seconds to run all queries: 15.584 seconds
	Number of clients running queries: 5
	Average number of queries per client: 10

Benchmark
	Average number of seconds to run all queries: 14.494 seconds
	Minimum number of seconds to run all queries: 13.220 seconds
	Maximum number of seconds to run all queries: 15.502 seconds
	Number of clients running queries: 10
	Average number of queries per client: 5

Benchmark
	Average number of seconds to run all queries: 13.468 seconds
	Minimum number of seconds to run all queries: 12.453 seconds
	Maximum number of seconds to run all queries: 14.397 seconds
	Number of clients running queries: 15
	Average number of queries per client: 3



top - 13:42:34 up  2:33,  1 user,  load average: 1.12, 0.29, 0.27
Tasks: 259 total,   2 running, 257 sleeping,   0 stopped,   0 zombie
%Cpu(s): 95.6 us,  1.5 sy,  0.0 ni,  0.0 id,  0.0 wa,  2.0 hi,  0.8 si,  0.0 st
MiB Mem :   5709.6 total,   1721.0 free,   2284.7 used,   1703.9 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3136.9 avail Mem 


top - 13:44:29 up  2:35,  1 user,  load average: 8.48, 3.03, 1.26
Tasks: 257 total,   2 running, 255 sleeping,   0 stopped,   0 zombie
%Cpu(s): 97.5 us,  1.0 sy,  0.0 ni,  0.0 id,  0.0 wa,  1.4 hi,  0.2 si,  0.0 st
MiB Mem :   5709.6 total,   1671.6 free,   2334.1 used,   1703.9 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3087.5 avail Mem 



To run a stress test with 20 threads, each executing the query 10 times, run the following command:
--------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=20 --iterations=10 --query="SELECT ID, Name, Age, Salary, Department, City, Country 
                                FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) AS R FROM loadtesttable) 
								AS ranked WHERE R <= 5;" --create-schema=loadtestdb --verbose

Benchmark
	Average number of seconds to run all queries: 58.641 seconds
	Minimum number of seconds to run all queries: 45.769 seconds
	Maximum number of seconds to run all queries: 97.248 seconds
	Number of clients running queries: 20
	Average number of queries per client: 1



top - 13:48:22 up  2:39,  1 user,  load average: 4.38, 4.33, 2.30
Tasks: 259 total,   3 running, 256 sleeping,   0 stopped,   0 zombie
%Cpu(s): 32.1 us, 62.0 sy,  0.0 ni,  0.0 id,  0.0 wa,  5.8 hi,  0.1 si,  0.0 st
MiB Mem :   5709.6 total,   1157.4 free,   2278.8 used,   2273.4 buff/cache
MiB Swap:   3072.0 total,   3072.0 free,      0.0 used.   3142.2 avail Mem 

top - 13:49:50 up  2:40,  1 user,  load average: 17.81, 8.94, 4.14
Tasks: 259 total,   2 running, 257 sleeping,   0 stopped,   0 zombie
%Cpu(s): 86.3 us,  5.3 sy,  0.0 ni,  4.4 id,  0.1 wa,  2.8 hi,  1.1 si,  0.0 st
MiB Mem :   5709.6 total,    727.8 free,   3759.4 used,   1222.4 buff/cache
MiB Swap:   3072.0 total,   3035.7 free,     36.2 used.   1676.3 avail Mem 


To run a benchmark test with 10 threads, each executing 1000 randomly generated queries with a mixed load type,
 run the following command:
 ----------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=10 --iterations=1 --number-of-queries=100000 --auto-generate-sql 
                                --auto-generate-sql-load-type=MIXED --verbose

Benchmark
	Average number of seconds to run all queries: 657.459 seconds
	Minimum number of seconds to run all queries: 657.459 seconds
	Maximum number of seconds to run all queries: 657.459 seconds
	Number of clients running queries: 10
	Average number of queries per client: 10000


[root@mysqlprod02 ~]# 

 top - 14:01:56 up  2:53,  1 user,  load average: 5.35, 9.72, 9.09
Tasks: 260 total,   1 running, 259 sleeping,   0 stopped,   0 zombie
%Cpu(s): 51.1 us, 30.5 sy,  0.0 ni, 14.6 id,  0.5 wa,  3.0 hi,  0.4 si,  0.0 st
MiB Mem :   5709.6 total,   2197.4 free,   3223.8 used,    288.4 buff/cache
MiB Swap:   3072.0 total,   1980.5 free,   1091.5 used.   2223.6 avail Mem 


top - 14:02:54 up  2:53,  1 user,  load average: 10.75, 10.58, 9.43
Tasks: 260 total,   2 running, 258 sleeping,   0 stopped,   0 zombie
%Cpu(s): 57.7 us, 29.9 sy,  0.0 ni,  9.4 id,  0.4 wa,  2.2 hi,  0.4 si,  0.0 st
MiB Mem :   5709.6 total,   2177.9 free,   3235.4 used,    296.3 buff/cache
MiB Swap:   3072.0 total,   1980.5 free,   1091.5 used.   2212.0 avail Mem 


In [ ]:

Run tests with custom innodb buffer settings
===========================================================

In [ ]:

[root@mysqlprod02 ~]# cat /etc/my.cnf | grep innodb

innodb-data-file-path = ibdata1:12M;ibdata2:20M:autoextend
innodb-flush-method = O_DIRECT_NO_FSYNC
innodb_redo_log_capacity = 536870912
innodb_buffer_pool_instances = 8
innodb_buffer_pool_size = 2G


mysql> show variables like '%innodb_flush_method%';
+---------------------+-------------------+
| Variable_name       | Value             |
+---------------------+-------------------+
| innodb_flush_method | O_DIRECT_NO_FSYNC |
+---------------------+-------------------+

mysql> show variables like '%innodb_buffer_pool%';
+-------------------------------------+----------------+
| Variable_name                       | Value          |
+-------------------------------------+----------------+
| innodb_buffer_pool_chunk_size       | 134217728      |
| innodb_buffer_pool_instances        | 8              |
| innodb_buffer_pool_size             | 2147483648     |
+-------------------------------------+----------------+

mysql> show variables like '%innodb_redo%';
+------------------------------+-----------+
| Variable_name                | Value     |
+------------------------------+-----------+
| innodb_redo_log_capacity     | 536870912 |
+------------------------------+-----------+


To run a concurrency test with 5, 10, and 15 threads, each executing 50 queries 5 times, run the following command
============================================================================================================================


[root@mysqlprod02 ~]# mysqlslap --concurrency=5,10,15 --iterations=5 --number-of-queries=50 
                                --query="SELECT ID, Name, Age, Salary, Department, City, Country FROM loadtesttable WHERE Name 
                                   like 'A%' AND Age BETWEEN 30 AND 40;" --create-schema=loadtestdb --verbose
                                   
Benchmark
	Average number of seconds to run all queries: 14.866 seconds
	Minimum number of seconds to run all queries: 13.194 seconds
	Maximum number of seconds to run all queries: 16.162 seconds
	Number of clients running queries: 5
	Average number of queries per client: 10

Benchmark
	Average number of seconds to run all queries: 14.178 seconds
	Minimum number of seconds to run all queries: 13.112 seconds
	Maximum number of seconds to run all queries: 14.953 seconds
	Number of clients running queries: 10
	Average number of queries per client: 5

Benchmark
	Average number of seconds to run all queries: 14.183 seconds
	Minimum number of seconds to run all queries: 12.172 seconds
	Maximum number of seconds to run all queries: 15.858 seconds
	Number of clients running queries: 15
	Average number of queries per client: 3



top - 14:36:56 up  3:28,  1 user,  load average: 2.37, 0.73, 2.73
Tasks: 260 total,   1 running, 259 sleeping,   0 stopped,   0 zombie
%Cpu(s): 96.3 us,  1.1 sy,  0.0 ni,  0.2 id,  0.0 wa,  2.2 hi,  0.3 si,  0.0 st
MiB Mem :   5709.6 total,   3352.8 free,   1850.3 used,    506.5 buff/cache
MiB Swap:   3072.0 total,   2501.7 free,    570.2 used.   3597.7 avail Mem 


top - 14:37:44 up  3:28,  1 user,  load average: 4.85, 1.65, 2.94
Tasks: 260 total,   2 running, 258 sleeping,   0 stopped,   0 zombie
%Cpu(s): 97.2 us,  1.5 sy,  0.0 ni,  0.0 id,  0.0 wa,  1.1 hi,  0.2 si,  0.0 st
MiB Mem :   5709.6 total,   3347.3 free,   1855.8 used,    506.5 buff/cache
MiB Swap:   3072.0 total,   2502.0 free,    570.0 used.   3592.2 avail Mem 


top - 14:39:30 up  3:30,  1 user,  load average: 12.87, 5.21, 4.07
Tasks: 261 total,   2 running, 259 sleeping,   0 stopped,   0 zombie
%Cpu(s): 93.4 us,  2.6 sy,  0.0 ni,  1.2 id,  0.0 wa,  2.1 hi,  0.7 si,  0.0 st
MiB Mem :   5709.6 total,   3298.7 free,   1904.4 used,    506.5 buff/cache
MiB Swap:   3072.0 total,   2502.0 free,    570.0 used.   3543.6 avail Mem 


To run a stress test with 20 threads, each executing the query 10 times, run the following command:
--------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=20 --iterations=10 --query="SELECT ID, Name, Age, Salary, Department, City, Country 
                                FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) AS R FROM loadtesttable) 
								AS ranked WHERE R <= 5;" --create-schema=loadtestdb --verbose


top - 14:42:23 up  3:33,  1 user,  load average: 5.98, 5.00, 4.20
Tasks: 258 total,   4 running, 254 sleeping,   0 stopped,   0 zombie
%Cpu(s): 62.6 us, 25.2 sy,  0.0 ni,  0.0 id,  0.0 wa, 11.4 hi,  0.8 si,  0.0 st
MiB Mem :   5709.6 total,   1681.2 free,   1907.7 used,   2120.6 buff/cache
MiB Swap:   3072.0 total,   2502.7 free,    569.2 used.   3539.9 avail Mem 

top - 14:43:05 up  3:34,  1 user,  load average: 14.76, 7.52, 5.09
Tasks: 258 total,   1 running, 257 sleeping,   0 stopped,   0 zombie
%Cpu(s): 95.2 us,  2.7 sy,  0.0 ni,  0.0 id,  0.0 wa,  1.9 hi,  0.2 si,  0.0 st
MiB Mem :   5709.6 total,   1315.2 free,   3094.2 used,   1300.2 buff/cache
MiB Swap:   3072.0 total,   2502.7 free,    569.2 used.   2353.2 avail Mem 


To run a benchmark test with 10 threads, each executing 1000 randomly generated queries with a mixed load type,
 run the following command:
 ----------------------------------------------------------------------------------------------------------------

[root@mysqlprod02 ~]# mysqlslap --concurrency=10 --iterations=1 --number-of-queries=100000 --auto-generate-sql 
                                --auto-generate-sql-load-type=MIXED --verbose


Benchmark
	Average number of seconds to run all queries: 622.510 seconds
	Minimum number of seconds to run all queries: 622.510 seconds
	Maximum number of seconds to run all queries: 622.510 seconds
	Number of clients running queries: 10
	Average number of queries per client: 10000


top - 15:04:07 up  3:55,  1 user,  load average: 2.89, 2.07, 5.35
Tasks: 260 total,   3 running, 257 sleeping,   0 stopped,   0 zombie
%Cpu(s): 45.8 us, 31.6 sy,  0.0 ni, 19.8 id,  0.8 wa,  1.9 hi,  0.1 si,  0.0 st
MiB Mem :   5709.6 total,   2148.1 free,   3288.1 used,    273.4 buff/cache
MiB Swap:   3072.0 total,   2174.7 free,    897.2 used.   2161.6 avail Mem 


top - 15:04:25 up  3:55,  1 user,  load average: 5.28, 2.66, 5.48
Tasks: 260 total,   1 running, 259 sleeping,   0 stopped,   0 zombie
%Cpu(s): 51.9 us, 30.2 sy,  0.0 ni, 14.9 id,  0.5 wa,  2.2 hi,  0.2 si,  0.0 st
MiB Mem :   5709.6 total,   2140.5 free,   3291.4 used,    277.7 buff/cache
MiB Swap:   3072.0 total,   2174.7 free,    897.2 used.   2158.2 avail Mem 


top - 15:08:27 up  3:59,  1 user,  load average: 15.70, 9.85, 7.79
Tasks: 261 total,   2 running, 259 sleeping,   0 stopped,   0 zombie
%Cpu(s): 64.5 us, 28.6 sy,  0.0 ni,  4.0 id,  0.1 wa,  2.5 hi,  0.4 si,  0.0 st
MiB Mem :   5709.6 total,   2084.5 free,   3317.5 used,    307.6 buff/cache
MiB Swap:   3072.0 total,   2175.0 free,    897.0 used.   2132.1 avail Mem 
